In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import sklearn
from random import random 
from random import sample 
from scipy.stats import norm
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.utils import class_weight
from sklearn import svm, datasets
import matplotlib 

matplotlib.rc('font', **{'size'   : 16})

In [ ]:
DATA = "../preprocessing/standardized_data.csv"
data = pd.read_csv(DATA,header=0)

## Again, reusing the misclassification definitions

In [ ]:
def gender_misclassification(gt, prediction): 
    """Given a vector of ground truth values `gt` and a vector of prediction values `gt`, 
    return a vector containing 1 if there has been gender misclassification and 0 otherwise. """
    n = len(gt)
    assert(len(prediction) == n)
    result = np.zeros(n)
    for i in range(n): 
        # If the ground truth or the result from the API is "unsure", no misclassification
        if gt[i] == 0 or prediction[i] == 0: 
            result[i] = 0
        # If the ground truth does not match the prediction, set the misclassification bit to one. 
        elif gt[i] != prediction[i]: 
            result[i] = 1
        else: 
            result[i] = 0
    return result

In [ ]:
def race_misclassification(gt, prediction): 
    """Given a vector of ground truth values `gt` and a vector of prediction values `gt`, 
    return a vector containing 1 if there has been race misclassification and 0 otherwise. """
    n = len(gt)
    assert(len(prediction) == n)
    result = np.zeros(n)
    for i in range(n): 
        # If the subjects is within a cornercase group or the prediction has outputed "not sure", then 
        # no misclassification
        if gt[i] in [0,5] or prediction[i] == 0 : 
            result[i] = 0
        # If the ground truth does not match the prediction, set the misclassification bit to one. 
        elif gt[i] != prediction[i]: 
            result[i] = 1
        else: 
            result[i] = 0
    return result

In [ ]:
def age_misclassification(gt, prediction): 
    """Given a vector of ground truth values `gt` and a vector of prediction values `gt`, 
    return a vector containing 1 if there has been age misclassification and 0 otherwise. """
    n = len(gt)
    assert(len(prediction) == n)
    result = np.zeros(n)
    
    for i in range(n): 
        tolerance = 0 
        # If the subject is a child, tolerance interval equals 5
        if gt[i] <= 10: 
            tolerance = 5
        # For teenagers and pre-young-adults, tolerance is 10
        elif gt[i] <= 25: 
            tolerance = 10
        # For the rest of the population, tolerance is 15
        else: 
            tolerance = 15
            
        if gt[i] - tolerance <= prediction[i] <= gt[i] + tolerance: 
            result[i] = 0
        else: 
            result[i] = 1
    return result

In [ ]:
def emotion_misclassification(gt, prediction): 
    """Given a vector of ground truth values `gt` and a vector of prediction values `gt`, 
    return a vector containing 1 if there has been emotion misclassification and 0 otherwise. """
    n = len(gt)
    assert(len(prediction) == n)
    result = np.zeros(n)
    
    for i in range(n): 
        # If dealing with one of the undefined emotions, set the misclassification bit to 0
        if gt[i] <= 0 or prediction[i] <= 0: 
            result[i] = 0 
        # If emotions don't match, set the misclassification bit to 1
        elif gt[i] != prediction[i]: 
            result[i] = 1
        else: 
            result[i] = 0
    return result

In [ ]:
def recall(estimator, X, y): 
    """ Calculate the recall of the estimator predicting on inputs X with output labels y"""

    prediction = estimator.predict(X)
    
    y = np.array(y)
    prediction = np.array(prediction)
        
    tp = 0
    fn = 0
    
    for i in range(y.size): 
        if y[i] == 1 and prediction[i] == 1:
            tp += 1
            continue 
        if y[i] == 1 and prediction[i] == 0: 
            fn += 1
            
    return (tp / (tp + fn))

In [ ]:
def precision(estimator, X, y):
    """ Calculate the precision of the estimator predicting on inputs X with output labels y"""
    
    prediction = estimator.predict(X)
    
    y = np.array(y)
    prediction = np.array(prediction)
    tp = 0
    fp = 0
    for i in range(y.size):
        if y[i] == 1 and prediction[i] == 1:
            tp += 1
            continue
        if y[i] == 0 and prediction[i] == 1:
            fp += 1
    return (tp / (tp + fp))

In [ ]:
def plot_scores(model, data=None, outputs=None): 
    """Given a classifier, display the cross-validation score on data (by default X_scaled) using 
    expected_cost, recall, precision and accuracy and scoring functions.""" 
    if data is None: 
        data = X_scaled
    if outputs is None: 
        outputs = y 
    recall_scores = cross_val_score(model, data, outputs, cv=5, scoring=recall)
    precision_scores = cross_val_score(model, data, outputs, cv=5, scoring=precision)
    accuracy_scores = cross_val_score(model, data, outputs, cv=5, scoring="accuracy")

    rec = sum(recall_scores)/10 
    prec = sum(precision_scores)/10
    acc = sum(accuracy_scores)/10
    
    df = pd.DataFrame((rec, prec, acc), index=["Recall", "Precision", "Accuracy"], columns=["Performance"])
    return df

# Gender Misclassification

In [ ]:
data_no_ai = data[data["Origin"] != "AI"]
gender_misclassification(data_no_ai["Gender"], data_no_ai["clarifai_gender"])

In [ ]:
cl = gender_misclassification(data_no_ai["Gender"], data_no_ai["clarifai_gender"]) == 1
ms = gender_misclassification(data_no_ai["Gender"], data_no_ai["microsoft_gender"]) == 1
am = gender_misclassification(data_no_ai["Gender"], data_no_ai["amazon_gender"]) == 1
fa = gender_misclassification(data_no_ai["Gender"], data_no_ai["face++_gender"]) == 1

In [ ]:
gender_misclass = (cl | ms | am |fa)
sum(gender_misclass)

In [ ]:
X = data_no_ai[["Race", "Age", "Gender", "Emotion"]]
y = [int(x) for x in gender_misclass]

In [ ]:
# One-hot encoding
X_eng = X.copy()

is_black = [int(x) for x in X_eng["Race"] == 2]
is_white = [int(x) for x in X_eng["Race"] == 4]
is_asian = [int(x) for x in X_eng["Race"] == 1]
is_latino = [int(x) for x in X_eng["Race"] == 3]
is_rest = [int(x) for x in X_eng["Race"] == 5]

is_happy = [int(x) for x in X_eng["Emotion"] == 5]
is_sad = [int(x) for x in X_eng["Emotion"] == 6]
is_calm = [int(x) for x in X_eng["Emotion"] == 2]
is_angry = [int(x) for x in X_eng["Emotion"] == 1]
is_fearful = [int(x) for x in X_eng["Emotion"] == 4]
is_surprised = [int(x) for x in X_eng["Emotion"] == 7]
is_disgusted = [int(x) for x in X_eng["Emotion"] == 3]

X_eng["is_black"] = is_black
X_eng["is_white"] = is_white
X_eng["is_asian"] = is_asian
X_eng["is_latino"] = is_latino
X_eng["is_rest"] = is_rest
X_eng["is_happy"] = is_happy
X_eng["is_sad"] = is_sad
X_eng["is_calm"] = is_calm
X_eng["is_angry"] = is_angry
X_eng["is_fearful"] = is_fearful
X_eng["is_surprised"] = is_surprised
X_eng["is_disgusted"] = is_disgusted

X = X_eng.drop(columns=["Race", "Emotion"])

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.15)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5))/5

In [ ]:
lr.fit(X_train_scaled, y_train)
prediction = lr.predict(X_test_scaled)

In [ ]:
sum(prediction)

In [ ]:
is_actually_misclass = np.array(y_test) == 1
is_not_misclass = np.array(y_test) == 0

In [ ]:
sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)

In [ ]:
sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)

In [ ]:
def balanced_accuracy(estimator, X, y):
    """ Calculate the precision of the estimator predicting on inputs X with output labels y"""
    
    prediction = np.array(estimator.predict(X))
    y = np.array(y)
    
    is_actually_misclass = y == 1
    is_not_misclass = y == 0
    
    acc_for_1 = sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)
    acc_for_0 = sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)
    
    return (acc_for_0 + acc_for_1) / 2

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
res = []
best_weight = None 
best_score = 0 
for w in np.linspace(1,30,50): 
    lr_tune = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:w})
    cv_score = sum(cross_val_score(lr_tune, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5
    if cv_score > best_score: 
        best_weight = w
        best_score = cv_score
    res.append(cv_score)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(np.linspace(1,30,50), res)
plt.xlabel("class_weight value")
plt.ylabel("Balanced accuracy")
#plt.savefig('training.svg', format='svg')
plt.show()

In [ ]:
print(best_weight)
print(best_score)

In [ ]:
lr_balanced = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:best_weight}) 
sum(cross_val_score(lr_balanced, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
lr_balanced.fit(X_train_scaled, y_train)
balanced_prediction = lr_balanced.predict(X_test_scaled)

In [ ]:
print("Accuracy for the 1-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 1] - 1))/sum(np.array(y_test) == 1))
print()
print("Accuracy for the 0-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 0] - 0))/sum(np.array(y_test) == 0))
print()
print("Balanced accuracy score: ")
print(balanced_accuracy(lr_balanced, X_test_scaled, y_test))
# Same as (acc_0 + acc_1)/2
print()
print("Weighted accuracy")
print(1 - sum(abs(balanced_prediction - y_test))/len(y_test))
print()
print("Recall")
print(recall(lr_balanced, X_test_scaled, y_test))
print()
print("Precision")
print(precision(lr_balanced, X_test_scaled, y_test))
print()

In [ ]:
print("We can see the most important factors for gender misclassification in out model are gender itself and age.")
lr_balanced.coef_

In [ ]:
lr_balanced.intercept_

In [ ]:
T = X.copy()

In [ ]:
list(zip(X.columns, lr_balanced.coef_[0]))

# Race Misclassification 

In [ ]:
data_no_ai = data[data["Origin"] != "AI"]

In [ ]:
race_misclass = race_misclassification(data_no_ai["Race"], data_no_ai["clarifai_race"]) == 1
sum(race_misclass)

In [ ]:
X = data_no_ai[["Race", "Age", "Gender", "Emotion"]]
y = [int(x) for x in race_misclass]

In [ ]:
# One-hot encoding
X_eng = X.copy()

is_black = [int(x) for x in X_eng["Race"] == 2]
is_white = [int(x) for x in X_eng["Race"] == 4]
is_asian = [int(x) for x in X_eng["Race"] == 1]
is_latino = [int(x) for x in X_eng["Race"] == 3]
is_rest = [int(x) for x in X_eng["Race"] == 5]

is_happy = [int(x) for x in X_eng["Emotion"] == 5]
is_sad = [int(x) for x in X_eng["Emotion"] == 6]
is_calm = [int(x) for x in X_eng["Emotion"] == 2]
is_angry = [int(x) for x in X_eng["Emotion"] == 1]
is_fearful = [int(x) for x in X_eng["Emotion"] == 4]
is_surprised = [int(x) for x in X_eng["Emotion"] == 7]
is_disgusted = [int(x) for x in X_eng["Emotion"] == 3]

X_eng["is_black"] = is_black
X_eng["is_white"] = is_white
X_eng["is_asian"] = is_asian
X_eng["is_latino"] = is_latino
X_eng["is_rest"] = is_rest
X_eng["is_happy"] = is_happy
X_eng["is_sad"] = is_sad
X_eng["is_calm"] = is_calm
X_eng["is_angry"] = is_angry
X_eng["is_fearful"] = is_fearful
X_eng["is_surprised"] = is_surprised
X_eng["is_disgusted"] = is_disgusted

X = X_eng.drop(columns=["Race", "Emotion"])

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.15)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5))/5

In [ ]:
lr.fit(X_train_scaled, y_train)
prediction = lr.predict(X_test_scaled)

In [ ]:
sum(prediction)

In [ ]:
is_actually_misclass = np.array(y_test) == 1
is_not_misclass = np.array(y_test) == 0

In [ ]:
sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)

In [ ]:
sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)

In [ ]:
def balanced_accuracy(estimator, X, y):
    """ Calculate the precision of the estimator predicting on inputs X with output labels y"""
    
    prediction = np.array(estimator.predict(X))
    y = np.array(y)
    
    is_actually_misclass = y == 1
    is_not_misclass = y == 0
    
    acc_for_1 = sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)
    acc_for_0 = sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)
    
    return (acc_for_0 + acc_for_1) / 2

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
res = []
best_weight = None 
best_score = 0 
for w in np.linspace(1,30,50): 
    lr_tune = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:w})
    cv_score = sum(cross_val_score(lr_tune, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5
    if cv_score > best_score: 
        best_weight = w
        best_score = cv_score
    res.append(cv_score)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(np.linspace(1,30,50), res)
plt.show()

In [ ]:
print(best_weight)
print(best_score)

In [ ]:
lr_balanced = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:best_weight}) 
sum(cross_val_score(lr_balanced, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
lr_balanced.fit(X_train_scaled, y_train)
balanced_prediction = lr_balanced.predict(X_test_scaled)

In [ ]:
print("Accuracy for the 1-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 1] - 1))/sum(np.array(y_test) == 1))
print()
print("Accuracy for the 0-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 0] - 0))/sum(np.array(y_test) == 0))
print()
print("Balanced accuracy score: ")
print(balanced_accuracy(lr_balanced, X_test_scaled, y_test))
# Same as (acc_0 + acc_1)/2
print()
print("Weighted accuracy")
print(1 - sum(abs(balanced_prediction - y_test))/len(y_test))
print()
print("Recall")
print(recall(lr_balanced, X_test_scaled, y_test))
print()
print("Precision")
print(precision(lr_balanced, X_test_scaled, y_test))
print()

In [ ]:
lr_balanced.coef_

In [ ]:
lr_balanced.intercept_

In [ ]:
T = X.copy()

In [ ]:
list(zip(X.columns, lr_balanced.coef_[0]))

# Age Misclassification Model  

In [ ]:
data_no_ai = data[data["Origin"] != "AI"]

In [ ]:
cl = age_misclassification(data["Age"], data["clarifai_age"]) == 1
ms = age_misclassification(data["Age"], data["microsoft_age"]) == 1
am = age_misclassification(data["Age"], data["amazon_age_average"]) == 1
fa = age_misclassification(data["Age"], data["face++_age"]) == 1

In [ ]:
age_misclass = (cl | ms | am |fa)
sum(age_misclass)

In [ ]:
len(data)

In [ ]:
X = data[["Race", "Age", "Gender", "Emotion"]]
y = [int(x) for x in age_misclass]

In [ ]:
# One-hot encoding
X_eng = X.copy()

is_black = [int(x) for x in X_eng["Race"] == 2]
is_white = [int(x) for x in X_eng["Race"] == 4]
is_asian = [int(x) for x in X_eng["Race"] == 1]
is_latino = [int(x) for x in X_eng["Race"] == 3]
is_rest = [int(x) for x in X_eng["Race"] == 5]

is_happy = [int(x) for x in X_eng["Emotion"] == 5]
is_sad = [int(x) for x in X_eng["Emotion"] == 6]
is_calm = [int(x) for x in X_eng["Emotion"] == 2]
is_angry = [int(x) for x in X_eng["Emotion"] == 1]
is_fearful = [int(x) for x in X_eng["Emotion"] == 4]
is_surprised = [int(x) for x in X_eng["Emotion"] == 7]
is_disgusted = [int(x) for x in X_eng["Emotion"] == 3]

X_eng["is_black"] = is_black
X_eng["is_white"] = is_white
X_eng["is_asian"] = is_asian
X_eng["is_latino"] = is_latino
X_eng["is_rest"] = is_rest
X_eng["is_happy"] = is_happy
X_eng["is_sad"] = is_sad
X_eng["is_calm"] = is_calm
X_eng["is_angry"] = is_angry
X_eng["is_fearful"] = is_fearful
X_eng["is_surprised"] = is_surprised
X_eng["is_disgusted"] = is_disgusted

X = X_eng.drop(columns=["Race", "Emotion"])

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.15)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5))/5

In [ ]:
lr.fit(X_train_scaled, y_train)
prediction = lr.predict(X_test_scaled)

In [ ]:
is_actually_misclass = np.array(y_test) == 1
is_not_misclass = np.array(y_test) == 0

In [ ]:
sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)

In [ ]:
sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)

In [ ]:
def balanced_accuracy(estimator, X, y):
    """ Calculate the precision of the estimator predicting on inputs X with output labels y"""
    
    prediction = np.array(estimator.predict(X))
    y = np.array(y)
    
    is_actually_misclass = y == 1
    is_not_misclass = y == 0
    
    acc_for_1 = sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)
    acc_for_0 = sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)
    
    return (acc_for_0 + acc_for_1) / 2

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
res = []
best_weight = None 
best_score = 0 
for w in np.linspace(1,3,50): 
    lr_tune = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:w})
    cv_score = sum(cross_val_score(lr_tune, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5
    if cv_score > best_score: 
        best_weight = w
        best_score = cv_score
    res.append(cv_score)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(np.linspace(1,3,50), res)
plt.show()

In [ ]:
print(best_weight)
print(best_score)

In [ ]:
lr_balanced = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:best_weight}) 
sum(cross_val_score(lr_balanced, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
lr_balanced.fit(X_train_scaled, y_train)
balanced_prediction = lr_balanced.predict(X_test_scaled)

In [ ]:
print("Accuracy for the 1-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 1] - 1))/sum(np.array(y_test) == 1))
print()
print("Accuracy for the 0-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 0] - 0))/sum(np.array(y_test) == 0))
print()
print("Balanced accuracy score: ")
print(balanced_accuracy(lr_balanced, X_test_scaled, y_test))
# Same as (acc_0 + acc_1)/2
print()
print("Weighted accuracy")
print(1 - sum(abs(balanced_prediction - y_test))/len(y_test))
print()
print("Recall")
print(recall(lr_balanced, X_test_scaled, y_test))
print()
print("Precision")
print(precision(lr_balanced, X_test_scaled, y_test))
print()

In [ ]:
lr_balanced.coef_

In [ ]:
lr_balanced.intercept_

In [ ]:
T = X.copy()

In [ ]:
list(zip(X.columns, lr_balanced.coef_[0]))

# Emotion Misclassification

In [ ]:
data_no_ai = data[data["Origin"] != "AI"]

In [ ]:
ms = emotion_misclassification(data_no_ai["Emotion"], data_no_ai["microsoft_emotion"]) == 1
am = emotion_misclassification(data_no_ai["Emotion"], data_no_ai["amazon_emotion"]) == 1
fa = emotion_misclassification(data_no_ai["Emotion"], data_no_ai["face++_emotion"]) == 1

In [ ]:
emotion_misclass = (ms | am | fa)

In [ ]:
X = data_no_ai[["Race", "Age", "Gender", "Emotion"]]
y = [int(x) for x in emotion_misclass]

In [ ]:
# One-hot encoding
X_eng = X.copy()

is_black = [int(x) for x in X_eng["Race"] == 2]
is_white = [int(x) for x in X_eng["Race"] == 4]
is_asian = [int(x) for x in X_eng["Race"] == 1]
is_latino = [int(x) for x in X_eng["Race"] == 3]
is_rest = [int(x) for x in X_eng["Race"] == 5]

is_happy = [int(x) for x in X_eng["Emotion"] == 5]
is_sad = [int(x) for x in X_eng["Emotion"] == 6]
is_calm = [int(x) for x in X_eng["Emotion"] == 2]
is_angry = [int(x) for x in X_eng["Emotion"] == 1]
is_fearful = [int(x) for x in X_eng["Emotion"] == 4]
is_surprised = [int(x) for x in X_eng["Emotion"] == 7]
is_disgusted = [int(x) for x in X_eng["Emotion"] == 3]

X_eng["is_black"] = is_black
X_eng["is_white"] = is_white
X_eng["is_asian"] = is_asian
X_eng["is_latino"] = is_latino
X_eng["is_rest"] = is_rest
X_eng["is_happy"] = is_happy
X_eng["is_sad"] = is_sad
X_eng["is_calm"] = is_calm
X_eng["is_angry"] = is_angry
X_eng["is_fearful"] = is_fearful
X_eng["is_surprised"] = is_surprised
X_eng["is_disgusted"] = is_disgusted

X = X_eng.drop(columns=["Race", "Emotion"])

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.15)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5))/5

In [ ]:
lr.fit(X_train_scaled, y_train)
prediction = lr.predict(X_test_scaled)

In [ ]:
def balanced_accuracy(estimator, X, y):
    """ Calculate the precision of the estimator predicting on inputs X with output labels y"""
    
    prediction = np.array(estimator.predict(X))
    y = np.array(y)
    
    is_actually_misclass = y == 1
    is_not_misclass = y == 0
    
    acc_for_1 = sum(prediction[is_actually_misclass] == 1)/sum(is_actually_misclass)
    acc_for_0 = sum(prediction[is_not_misclass] == 0)/sum(is_not_misclass)
    
    return (acc_for_0 + acc_for_1) / 2

In [ ]:
lr = LogisticRegression(solver='lbfgs') 

In [ ]:
sum(cross_val_score(lr, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
res = []
best_weight = None 
best_score = 0 
for w in np.linspace(1,30,50): 
    lr_tune = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:w})
    cv_score = sum(cross_val_score(lr_tune, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5
    if cv_score > best_score: 
        best_weight = w
        best_score = cv_score
    res.append(cv_score)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(np.linspace(1,30,50), res)
plt.xlabel("class_weight value")
plt.ylabel("Balanced accuracy")
#plt.savefig('training.svg', format='svg')
plt.show()

In [ ]:
print(best_weight)
print(best_score)

In [ ]:
lr_balanced = LogisticRegression(solver='lbfgs', class_weight={0:1, 1:best_weight}) 
sum(cross_val_score(lr_balanced, X_train_scaled, y_train, cv=5, scoring=balanced_accuracy))/5

In [ ]:
lr_balanced.fit(X_train_scaled, y_train)
balanced_prediction = lr_balanced.predict(X_test_scaled)

In [ ]:
print("Accuracy for the 1-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 1] - 1))/sum(np.array(y_test) == 1))
print()
print("Accuracy for the 0-class")
print(1 - sum(abs(balanced_prediction[np.array(y_test) == 0] - 0))/sum(np.array(y_test) == 0))
print()
print("Balanced accuracy score: ")
print(balanced_accuracy(lr_balanced, X_test_scaled, y_test))
# Same as (acc_0 + acc_1)/2
print()
print("Weighted accuracy")
print(1 - sum(abs(balanced_prediction - y_test))/len(y_test))
print()
print("Recall")
print(recall(lr_balanced, X_test_scaled, y_test))
print()
print("Precision")
print(precision(lr_balanced, X_test_scaled, y_test))
print()